### 1. 使用np.exp()实现sigmoid function

In [1]:
import numpy as np

def basic_sigmoid(x):
    '''
   计算sigmoid函数 
    '''
    # 开始
    s = 1/(1+np.exp(-x))
    # 结束
    return s

In [2]:
basic_sigmoid(-4)

0.01798620996209156

In [3]:
basic_sigmoid(np.array([-3, 0, 5]))

array([0.04742587, 0.5       , 0.99330715])

### 实现Sigmoid gradient (梯度)

完成sigmoid的梯度函数，用它去计算sigmoid相对于其输入x的梯度 $\sigma'(x)=\sigma(x)(1-\sigma(x))$

In [4]:
def sigmoid_derivative(x):
    '''
   计算sigmoid function函数相对于其输入x的梯度 (也称为斜率或者导数)
    '''
    # 开始
    ds = basic_sigmoid(x)*(1-basic_sigmoid(x))
    # 结束
    return ds

In [5]:
sigmoid_derivative(-4)

0.017662706213291118

In [6]:
sigmoid_derivative(np.array([-3, 0, 5, 20]))

array([4.51766597e-02, 2.50000000e-01, 6.64805667e-03, 2.06115369e-09])

### 3.手写数字'0'和'1'分类的logistic回归

#### 要求
实现一个学习算法的整体结构：

* 获取并定义模型输入
* 初始化参数
* 计算成本函数及其梯度
* 使用优化算法(梯度下降).
  * 计算当前损失(正向传播)。
  * 计算当前梯度(反向传摄)。
  * 更新参数 (梯度下降)
  
请实现相关函数，其中读取数据不需要实现。并在主模型函数中场写逻辑

#### 读取数据
1. 按照向量化伪代码实现的形状要求将样本数据进行转换 
2. 标准化数据处理

In [1]:
import torch
import torchvision
import numpy as np
from matplotlib import pyplot as plt
from torchvision import datasets, transforms

In [2]:
test_data = datasets.MNIST(root="./data/", train=False, download=True)

In [3]:
test_data[1][0]

In [4]:
test_data[1][1]

2

In [5]:
test_data = datasets.MNIST(root="./data/", train=False, download=True, transform=transforms.ToTensor())

In [6]:
test_data[1][0]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.4549, 0.4902, 0.6706, 1.0000, 1.0000, 0.5882,
          0.3647, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [12]:
train_X = []
test_X = []
train_Y = []
test_Y = []
class_1 = 1
class_0 = 0

In [13]:
for i in range(10000):
    if test_data[i][1]==class_1 and i < 8000:
        train_X.append(test_data[i][0].numpy().reshape(-1,))
        train_Y.append(np.array([1]))
    if test_data[i][1]==class_0 and i < 8000:
        train_X.append(test_data[i][0].numpy().reshape(-1,))
        train_Y.append(np.array([0]))
    if test_data[i][1]==class_1 and i >= 8000:
        test_X.append(test_data[i][0].numpy().reshape(-1,))
        test_Y.append(np.array([1]))
    if test_data[i][1]==class_0 and i >= 8000:
        test_X.append(test_data[i][0].numpy().reshape(-1,))
        test_Y.append(np.array([0]))

In [14]:
train_X = np.array(train_X).T
test_X = np.array(test_X).T
train_Y = np.array(train_Y).T
test_Y = np.array(test_Y).T
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

(784, 1678) (784, 437) (1, 1678) (1, 437)


### 初始化网络参数

In [15]:
def initialize_with_zeros(shape):
    """
    创建一个形状为(1,shape)的w参数和b=0
    """
    ### 开始
    w = np.zeros((1,shape))
    b = 0
    ### 结束
    return w,b

### 前向和反向传播
根据损失函数、前后传播向量化代码

In [16]:
def propagate(w, b, X, Y):
    """
    参数：网络参数w(1*shape),b(1)  和    数据X(shape*N_sample),Y(1*N_sample)
    Return:
    损失cost，参数w的梯度dw(1*shape)、参数b的梯度db(1)
    """
    # -------------前向传播--------------
    ### 开始
    z = np.dot(w, X) + b
    Y_hat = basic_sigmoid(z)
    cost = np.mean(-Y*np.log(Y_hat) - (1-Y)*np.log(1-Y_hat))
    ### 结束
    
    # -------------反向传播--------------
    ### 开始
    dz = Y_hat-Y
    dw = np.dot(Y_hat-Y,X.T)/X.shape[1]
    db = np.mean(dz)
    ### 结束
    
    grads = {'dw':dw, 'db':db}
    return grads, cost

### 优化过程
实现优化函数,全局的参数随着$w,b$对损失$J$进行优化改变。<br>
给定学习率$\alpha$下，对参数$\theta$实行$\theta=\theta-\alpha d\theta $。

In [17]:
def optimize(w, b, X, Y, num_iterations, learning_rate):
    """
    参数:
    权重w，偏置b，特征X，目标值Y，总迭代次数num_iterations，学习率learning_rate
    Returns：
    params：更新后的参数字典
    grads：梯度
    costs：每次迭代的cost构成的列表
    """
    costs = []
    for i in range(num_iterations):
        # ---------调用梯度计算的函数，并据此更新参数----------
        ### 开始
        grads, cost = propagate(w, b, X, Y)
        w = w - learning_rate*grads['dw']
        b = b - learning_rate*grads['db']
        
        ### 结束        
        
        costs.append(cost)
        if i % 10 == 0:
            print('损失函数 %i:%f' %(i, cost))
        
    params = {'w':w, 'b':b}
    return params, costs  

### 预测函数（不需要实现，后面调用即可）
利用得出的参数采进行测试得出准确率

In [18]:
def predict(w, b, X):
    """
    利用训练好的参数w和b和输入X
    此时X可以为训练集(样本内预测),也可以为测试集(样本外预测)
    return：预测结果
    """
    # 计算结果
    A = basic_sigmoid(np.dot(w, X) + b)
    Y_pre = np.where(A>0.5, 1, 0)
    
    return Y_pre

### 整体逻辑实现

In [19]:
 def model(X_train, Y_train, X_test, Y_test, num_iterations=100, learning_rate=0.01):
    ### 开始
    # 调用函数：初始化参数
    w,b = initialize_with_zeros(X_train.shape[0])
    # 调用函数：梯度下降法优化参数
    params, costs  = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)
    # 获取训练参数
    w = params['w']
    b = params['b']
    # 预测结果
    Y_pre_train = predict(w, b, train_X)    
    Y_pre_test = predict(w, b, test_X)    
    ### 结束
    
    # 打印准确率
    print('--'*50)
    print('训练集准确率%f'%(np.mean(Y_pre_train==Y_train)*100))
    print('测试集准确率%f'%(np.mean(Y_pre_test==Y_test)*100))  
    
    answers = {'costs':costs,
        'Y_prediction_train':Y_pre_train,
        'Y_prediction_test':Y_pre_test,
        'w':w, 'b':b,
        'num_iterations':num_iterations,
        'learning_rate':learning_rate}
    
    return answers

In [20]:
d = model(train_X, train_Y, test_X, test_Y, num_iterations = 200, learning_rate = 0.05)

损失函数 0:0.693147
损失函数 10:0.184616
损失函数 20:0.108454
损失函数 30:0.078364
损失函数 40:0.062121
损失函数 50:0.051884
损失函数 60:0.044806
损失函数 70:0.039600
损失函数 80:0.035598
损失函数 90:0.032418
损失函数 100:0.029826
损失函数 110:0.027668
损失函数 120:0.025841
损失函数 130:0.024272
损失函数 140:0.022910
损失函数 150:0.021714
损失函数 160:0.020655
损失函数 170:0.019710
损失函数 180:0.018862
损失函数 190:0.018094
----------------------------------------------------------------------------------------------------
训练集准确率99.940405
测试集准确率99.542334
